In [1]:
import librosa as lr
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import logging

from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot  


class VoiceSamples(Dataset):
    
    def __init__(self, core_name, samples_path=None, Automatic=None):
        
        self.Log = logging.getLogger()
        logging.basicConfig(level=logging.INFO)
        
        self.noiseThreshold = 1
        
        self.core_name = core_name
        self.samples_path = samples_path
        
        self.soundSamples = []
        self.sampleRate = []
        self.path = []
        
        self.chopedSamples = []
        self.chopedSr = []
        
        self.tensorMelgrams = []
        
        
        self.info = " VoiceSamples Object successfully created "
        self.Log.info(self.info)
        
        
        if Automatic:
            self.LoadSoundSamples()
            self.ChopToOneSecFragments()
            self.ChopedSignalsToTenosor()
        
    def __len__(self):
        return len(self.tensorMelgrams)
    
    def __getitem__(self, idx):
        if self.tensorMelgrams:
            return self.tensorMelgrams[idx]

    def LoadSoundSamples(self):
    
        n = 1

        while(True):
            try:
                if  self.samples_path:
                    path =  self.samples_path + self.core_name + str(n)
                else:
                    path = self.core_name + str(n)

                soundSample, sampleRate = lr.load(path)

                n += 1
                self.soundSamples.append(soundSample)
                self.sampleRate.append(sampleRate) 
                self.path.append(path)

                self.info = " Sample : " + path + " : successfully added"
                self.Log.info(self.info)

            except FileNotFoundError:
                if self.soundSamples:
                    self.info = "That's the end of database : " + str(n-1) + " : Samples added"
                    self.Log.info(self.info)
                    n = 0
                    
                    return self.soundSamples, self.sampleRate, self.path

                else:
                    self.Log.exception("Files are missing")
                    n = 0

                break

            except Exception as ex:      
                self.Log.exception("Unexpected error")
                break
        
    def getSoundSample(self, idx):
        return self.soundSamples[idx], self.sampleRate[idx]
    
    def getSoundSampleLen(self):
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                return len(self.soundSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
    
    def ChopToOneSecFragments(self):
        
        # TODO: make shure user goes step by step 
        
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                for idx in range(len(self.soundSamples)):
                    
                    soundSample = self.soundSamples[idx]
                    sr = self.sampleRate[idx]
                    
                    frag_max = math.trunc(len(soundSample)/float(sr))
                    step = math.trunc(sr/2);
                    last_sample = len(soundSample)

                    for frag in range(frag_max*2):
                        start = step * frag
                        stop = start + sr
                        if sr<len(soundSample):
                            if self.checkIfNotNoise(soundSample[start:stop]):
                                self.chopedSamples.append(soundSample[start:stop])
                                self.chopedSr.append(sr)
                                self.info = self.path[idx] + " : " + str(frag+1) + " : successfully choped"
                                self.Log.info(self.info)
                            else:
                                self.info = self.path[idx] + " : " + str(frag+1) + " : NOISE!"
                                self.Log.info(self.info)
                        else:
                            self.Log.warning("Something went wrong")
                            
                    if self.checkIfNotNoise(soundSample[last_sample-sr:last_sample]):
                         # incuding samples cuted by math.trunc() 
                        self.chopedSamples.append(soundSample[last_sample-sr:last_sample])
                        self.chopedSr.append(sr)
                        self.info = self.path[idx] +  " : "  + str(frag_max*2+1) + " : successfully choped"
                        self.Log.info(self.info)
                    else:
                        self.info = self.path[idx] + " : "  + str(frag+1) + " : NOISE!"
                        self.Log.info(self.info)
                
                if self.chopedSamples:
                    self.Log.info("Sucessfully choped all loaded signals and eliminated the noise!")
                    return self.chopedSamples, self.chopedSr 
                    
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
            
    def getChoped(self, idx):
        return self.chopedSamples[idx], self.chopedSr[idx]
        
    def getChopedLen(self):
        try:
            if len(self.chopedSamples) == len(self.chopedSr):
                    return len(self.chopedSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
            
        
    def ChopedSignalsToTenosor(self):
        
        # TODO: make shure user goes step by step 
        
        try:
        
            if len(self.chopedSamples) == len(self.chopedSr):
                for idx in range(len(self.chopedSamples)):

                    # hop length adjusted
                    STFT_signal = np.abs(lr.stft(self.chopedSamples[idx], n_fft = 512, hop_length = round(self.chopedSr[idx]/256))) 
                    STFT_signal = lr.power_to_db(STFT_signal**2,ref=np.max)

                    Melgram = STFT_signal[0:256,0:256]
                    TMelgram = torch.tensor(Melgram)
                    self.tensorMelgrams.append(TMelgram)
                    
                    self.info = " " + self.samples_path +  " : ChopedSample " + str(idx) + " : " + " : converted to tensor"
                    self.Log.info(self.info)
                
                if self.tensorMelgrams:
                    self.Log.info("Sucessfully converted all ChopedSamples to Tensors!")
                    return self.tensorMelgrams
                
            else:
                self.Log.warning("Lists: chopedSamples and chopedSr are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
                
    
    
    def checkIfNotNoise(self, chopedSample):
    
        chopedSamplePow2 = []

        for n in range(len(chopedSample)):
            chopedSamplePow2.append(chopedSample[n]**2)
        sk = sum(chopedSamplePow2)
        if sk > self.noiseThreshold:
            return True 
        else:
            return False

In [2]:
# Use torch.save(tensor, 'file.pt') and torch.load('file.pt')

class VoiceSamplesInput():
    def __init__(self):
        
        self.vsKrystian = VoiceSamples("vsKrystian", samples_path="database/Krystian/" , Automatic=True)
        self.vsNicia = VoiceSamples("vsNicia", samples_path="database/Nicia/" , Automatic=True)

        self.targetKrystian = torch.tensor([[float(1),float(0)]])
        self.targetNicia = torch.tensor([[float(0),float(1)]])
        
        
    def __getitem__(self, idx):
        if idx % 2 == 0:
            return self.vsKrystian[int(idx/2)] ,  self.targetKrystian
        else:
            return self.vsNicia[int((idx+1)/2)] , self.targetNicia
    
    def __len__(self):
        if len(self.vsKrystian) <= len(self.vsNicia):
            return len(self.vsKrystian) * 2
        else:
            return len(self.vsNicia) * 2
            




In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VoiceRecogModel(nn.Module):

    def __init__(self):
        super(VoiceRecogModel, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.conv2 = nn.Conv2d(10, 20, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(20*62*62, 2000)  # ?? from image dimension
        self.fc2 = nn.Linear(2000, 300)
        self.fc3 = nn.Linear(300, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
import torch.optim as optim

# create dataset : loading and processing samples to tensors
vsInput = VoiceSamplesInput()


INFO:root: VoiceSamples Object successfully created 
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root: Sample : database/Krystian/vsKrystian1 : successfully added
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root: Sample : database/Krystian/vsKrystian2 : successfully added
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
INFO:root: Sample : database/Krystian/vsKrystian3 : successfully added
/home/krys/.local/lib/python3.7/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.

INFO:root:database/Krystian/vsKrystian2 : 58 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 59 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 60 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 61 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 62 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 63 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 64 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 65 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 66 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 67 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 68 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 69 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 70 : NOISE!
INFO:root:database/Krystian/vsKrystian2 : 71 : successfully choped
INFO:root:database/Krystian/vsKrystian2 : 72 : successfully choped
INFO:roo

INFO:root:database/Krystian/vsKrystian4 : 5 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 6 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 7 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 8 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 9 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 10 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 11 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 12 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 13 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 14 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 15 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 16 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 17 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 18 : successfully choped
INFO:root:database/Krystian/vsKrystian4 : 19 : successfully choped


INFO:root:database/Krystian/vsKrystian6 : 40 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 41 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 42 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 43 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 44 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 45 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 46 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 47 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 48 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 49 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 50 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 51 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 52 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 53 : successfully choped
INFO:root:database/Krystian/vsKrystian6 : 54 : NOISE!
INFO:roo

INFO:root:database/Krystian/vsKrystian7 : 110 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 111 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 112 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 113 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 114 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 115 : NOISE!
INFO:root:database/Krystian/vsKrystian7 : 116 : successfully choped
INFO:root:database/Krystian/vsKrystian7 : 117 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 1 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 2 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 3 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 4 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 5 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 6 : successfully choped
INFO:root:database/Krystian/vsKrystian8 : 7 : successfully choped
INFO:ro

...

INFO:root: database/Nicia/ : ChopedSample 367 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 368 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 369 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 370 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 371 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 372 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 373 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 374 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 375 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 376 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 377 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 378 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 379 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 380 :  : converted to tensor
INFO:r

INFO:root: database/Nicia/ : ChopedSample 483 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 484 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 485 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 486 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 487 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 488 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 489 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 490 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 491 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 492 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 493 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 494 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 495 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 496 :  : converted to tensor
INFO:r

INFO:root: database/Nicia/ : ChopedSample 599 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 600 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 601 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 602 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 603 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 604 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 605 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 606 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 607 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 608 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 609 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 610 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 611 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 612 :  : converted to tensor
INFO:r

INFO:root: database/Nicia/ : ChopedSample 715 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 716 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 717 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 718 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 719 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 720 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 721 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 722 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 723 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 724 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 725 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 726 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 727 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 728 :  : converted to tensor
INFO:r

INFO:root: database/Nicia/ : ChopedSample 831 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 832 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 833 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 834 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 835 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 836 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 837 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 838 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 839 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 840 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 841 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 842 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 843 :  : converted to tensor
INFO:root: database/Nicia/ : ChopedSample 844 :  : converted to tensor
INFO:r

In [12]:
# create net from VoiceRecogModel
net = VoiceRecogModel()

# loss function (using function implemented in pytorch)
criterion = nn.MSELoss()

# create your optimizer (basic optimizer)
# setting learning rate 
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [13]:
import logging
logging.basicConfig(level=logging.INFO)
trainLog = logging.getLogger()

# Training Loop

# How many times learn on the same dataset
epoch = 1
for i in range(epoch):   
    for k in range(len(vsInput)):  
        try:
            vs, target = vsInput[k]

            optimizer.zero_grad()   # zero the gradient buffers
            input = vs.view(-1,1,256,256)
            output = net(input)

            loss = criterion(output, target)

            loss.backward()
            optimizer.step()    # Does the update


            info = "Training " + str(k)+"/"+str(len(vsInput))+" done"

            print("\n")
            print(target)
            print(output)
            print(loss)
            print(info)
        except Exception as e:
            print(e)
            pass



tensor([[1., 0.]])
tensor([[ 0.2650, -0.2483]], grad_fn=<AddmmBackward>)
tensor(0.3009, grad_fn=<MseLossBackward>)
Training 0/1412 done


tensor([[0., 1.]])
tensor([[6.2864, 0.9393]], grad_fn=<AddmmBackward>)
tensor(19.7612, grad_fn=<MseLossBackward>)
Training 1/1412 done


tensor([[1., 0.]])
tensor([[-0.3570, -0.0792]], grad_fn=<AddmmBackward>)
tensor(0.9239, grad_fn=<MseLossBackward>)
Training 2/1412 done


tensor([[0., 1.]])
tensor([[ 1.2121, -0.1732]], grad_fn=<AddmmBackward>)
tensor(1.4228, grad_fn=<MseLossBackward>)
Training 3/1412 done


tensor([[1., 0.]])
tensor([[-0.0202,  0.3052]], grad_fn=<AddmmBackward>)
tensor(0.5670, grad_fn=<MseLossBackward>)
Training 4/1412 done


tensor([[0., 1.]])
tensor([[0.0281, 0.1592]], grad_fn=<AddmmBackward>)
tensor(0.3539, grad_fn=<MseLossBackward>)
Training 5/1412 done


tensor([[1., 0.]])
tensor([[0.0428, 0.4387]], grad_fn=<AddmmBackward>)
tensor(0.5543, grad_fn=<MseLossBackward>)
Training 6/1412 done


tensor([[0., 1.]])
tensor([[0.0869, 0



tensor([[1., 0.]])
tensor([[0.2364, 0.3308]], grad_fn=<AddmmBackward>)
tensor(0.3462, grad_fn=<MseLossBackward>)
Training 60/1412 done


tensor([[0., 1.]])
tensor([[0.6088, 0.7342]], grad_fn=<AddmmBackward>)
tensor(0.2206, grad_fn=<MseLossBackward>)
Training 61/1412 done


tensor([[1., 0.]])
tensor([[0.2725, 0.4044]], grad_fn=<AddmmBackward>)
tensor(0.3464, grad_fn=<MseLossBackward>)
Training 62/1412 done


tensor([[0., 1.]])
tensor([[0.3535, 0.5039]], grad_fn=<AddmmBackward>)
tensor(0.1855, grad_fn=<MseLossBackward>)
Training 63/1412 done


tensor([[1., 0.]])
tensor([[0.5597, 0.8120]], grad_fn=<AddmmBackward>)
tensor(0.4266, grad_fn=<MseLossBackward>)
Training 64/1412 done


tensor([[0., 1.]])
tensor([[0.1363, 0.2209]], grad_fn=<AddmmBackward>)
tensor(0.3128, grad_fn=<MseLossBackward>)
Training 65/1412 done


tensor([[1., 0.]])
tensor([[0.5473, 0.5930]], grad_fn=<AddmmBackward>)
tensor(0.2783, grad_fn=<MseLossBackward>)
Training 66/1412 done


tensor([[0., 1.]])
tensor([[0.4682, 0.4



tensor([[1., 0.]])
tensor([[0.3988, 0.5026]], grad_fn=<AddmmBackward>)
tensor(0.3071, grad_fn=<MseLossBackward>)
Training 120/1412 done


tensor([[0., 1.]])
tensor([[0.4855, 0.5781]], grad_fn=<AddmmBackward>)
tensor(0.2069, grad_fn=<MseLossBackward>)
Training 121/1412 done


tensor([[1., 0.]])
tensor([[0.3637, 0.4783]], grad_fn=<AddmmBackward>)
tensor(0.3168, grad_fn=<MseLossBackward>)
Training 122/1412 done


tensor([[0., 1.]])
tensor([[0.4883, 0.5474]], grad_fn=<AddmmBackward>)
tensor(0.2217, grad_fn=<MseLossBackward>)
Training 123/1412 done


tensor([[1., 0.]])
tensor([[0.4031, 0.4988]], grad_fn=<AddmmBackward>)
tensor(0.3026, grad_fn=<MseLossBackward>)
Training 124/1412 done


tensor([[0., 1.]])
tensor([[0.4883, 0.5418]], grad_fn=<AddmmBackward>)
tensor(0.2242, grad_fn=<MseLossBackward>)
Training 125/1412 done


tensor([[1., 0.]])
tensor([[0.4122, 0.4989]], grad_fn=<AddmmBackward>)
tensor(0.2972, grad_fn=<MseLossBackward>)
Training 126/1412 done


tensor([[0., 1.]])
tensor([[0.49



tensor([[1., 0.]])
tensor([[0.4345, 0.4817]], grad_fn=<AddmmBackward>)
tensor(0.2759, grad_fn=<MseLossBackward>)
Training 180/1412 done


tensor([[0., 1.]])
tensor([[0.5540, 0.5227]], grad_fn=<AddmmBackward>)
tensor(0.2674, grad_fn=<MseLossBackward>)
Training 181/1412 done


tensor([[1., 0.]])
tensor([[0.3923, 0.4528]], grad_fn=<AddmmBackward>)
tensor(0.2872, grad_fn=<MseLossBackward>)
Training 182/1412 done


tensor([[0., 1.]])
tensor([[0.4879, 0.4765]], grad_fn=<AddmmBackward>)
tensor(0.2560, grad_fn=<MseLossBackward>)
Training 183/1412 done


tensor([[1., 0.]])
tensor([[0.4193, 0.4957]], grad_fn=<AddmmBackward>)
tensor(0.2915, grad_fn=<MseLossBackward>)
Training 184/1412 done


tensor([[0., 1.]])
tensor([[0.5256, 0.5372]], grad_fn=<AddmmBackward>)
tensor(0.2452, grad_fn=<MseLossBackward>)
Training 185/1412 done


tensor([[1., 0.]])
tensor([[0.3949, 0.4715]], grad_fn=<AddmmBackward>)
tensor(0.2942, grad_fn=<MseLossBackward>)
Training 186/1412 done


tensor([[0., 1.]])
tensor([[0.53



tensor([[1., 0.]])
tensor([[0.3999, 0.5103]], grad_fn=<AddmmBackward>)
tensor(0.3102, grad_fn=<MseLossBackward>)
Training 240/1412 done


tensor([[0., 1.]])
tensor([[0.5068, 0.5710]], grad_fn=<AddmmBackward>)
tensor(0.2204, grad_fn=<MseLossBackward>)
Training 241/1412 done


tensor([[1., 0.]])
tensor([[0.3566, 0.4616]], grad_fn=<AddmmBackward>)
tensor(0.3136, grad_fn=<MseLossBackward>)
Training 242/1412 done


tensor([[0., 1.]])
tensor([[0.4977, 0.5656]], grad_fn=<AddmmBackward>)
tensor(0.2182, grad_fn=<MseLossBackward>)
Training 243/1412 done


tensor([[1., 0.]])
tensor([[0.3486, 0.4542]], grad_fn=<AddmmBackward>)
tensor(0.3153, grad_fn=<MseLossBackward>)
Training 244/1412 done


tensor([[0., 1.]])
tensor([[0.5077, 0.6376]], grad_fn=<AddmmBackward>)
tensor(0.1945, grad_fn=<MseLossBackward>)
Training 245/1412 done


tensor([[1., 0.]])
tensor([[0.3801, 0.4451]], grad_fn=<AddmmBackward>)
tensor(0.2912, grad_fn=<MseLossBackward>)
Training 246/1412 done


tensor([[0., 1.]])
tensor([[0.44

...



tensor([[1., 0.]])
tensor([[ 1.2979, -0.0537]], grad_fn=<AddmmBackward>)
tensor(0.0458, grad_fn=<MseLossBackward>)
Training 1316/1412 done


tensor([[0., 1.]])
tensor([[-0.0776,  0.9272]], grad_fn=<AddmmBackward>)
tensor(0.0057, grad_fn=<MseLossBackward>)
Training 1317/1412 done


tensor([[1., 0.]])
tensor([[0.9178, 0.0062]], grad_fn=<AddmmBackward>)
tensor(0.0034, grad_fn=<MseLossBackward>)
Training 1318/1412 done


tensor([[0., 1.]])
tensor([[0.0272, 0.8768]], grad_fn=<AddmmBackward>)
tensor(0.0080, grad_fn=<MseLossBackward>)
Training 1319/1412 done


tensor([[1., 0.]])
tensor([[1.0088, 0.0126]], grad_fn=<AddmmBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)
Training 1320/1412 done


tensor([[0., 1.]])
tensor([[0.0190, 1.1832]], grad_fn=<AddmmBackward>)
tensor(0.0170, grad_fn=<MseLossBackward>)
Training 1321/1412 done


tensor([[1., 0.]])
tensor([[ 1.0473, -0.0880]], grad_fn=<AddmmBackward>)
tensor(0.0050, grad_fn=<MseLossBackward>)
Training 1322/1412 done


tensor([[0., 1.]])




tensor([[0., 1.]])
tensor([[0.2660, 0.4580]], grad_fn=<AddmmBackward>)
tensor(0.1823, grad_fn=<MseLossBackward>)
Training 1375/1412 done


tensor([[1., 0.]])
tensor([[1.0592, 0.0506]], grad_fn=<AddmmBackward>)
tensor(0.0030, grad_fn=<MseLossBackward>)
Training 1376/1412 done


tensor([[0., 1.]])
tensor([[0.2526, 0.6374]], grad_fn=<AddmmBackward>)
tensor(0.0976, grad_fn=<MseLossBackward>)
Training 1377/1412 done


tensor([[1., 0.]])
tensor([[1.0310, 0.2020]], grad_fn=<AddmmBackward>)
tensor(0.0209, grad_fn=<MseLossBackward>)
Training 1378/1412 done


tensor([[0., 1.]])
tensor([[0.0473, 0.8629]], grad_fn=<AddmmBackward>)
tensor(0.0105, grad_fn=<MseLossBackward>)
Training 1379/1412 done


tensor([[1., 0.]])
tensor([[0.9677, 0.1416]], grad_fn=<AddmmBackward>)
tensor(0.0106, grad_fn=<MseLossBackward>)
Training 1380/1412 done


tensor([[0., 1.]])
tensor([[0.0992, 0.7259]], grad_fn=<AddmmBackward>)
tensor(0.0425, grad_fn=<MseLossBackward>)
Training 1381/1412 done


tensor([[1., 0.]])
tensor



tensor([[0., 1.]])
tensor([[0.2670, 0.6840]], grad_fn=<AddmmBackward>)
tensor(0.0856, grad_fn=<MseLossBackward>)
Training 23/1412 done


tensor([[1., 0.]])
tensor([[0.8993, 0.0028]], grad_fn=<AddmmBackward>)
tensor(0.0051, grad_fn=<MseLossBackward>)
Training 24/1412 done


tensor([[0., 1.]])
tensor([[0.2922, 0.8056]], grad_fn=<AddmmBackward>)
tensor(0.0616, grad_fn=<MseLossBackward>)
Training 25/1412 done


tensor([[1., 0.]])
tensor([[0.6873, 0.1901]], grad_fn=<AddmmBackward>)
tensor(0.0670, grad_fn=<MseLossBackward>)
Training 26/1412 done


tensor([[0., 1.]])
tensor([[0.2729, 0.7225]], grad_fn=<AddmmBackward>)
tensor(0.0757, grad_fn=<MseLossBackward>)
Training 27/1412 done


tensor([[1., 0.]])
tensor([[0.6365, 0.1466]], grad_fn=<AddmmBackward>)
tensor(0.0768, grad_fn=<MseLossBackward>)
Training 28/1412 done


tensor([[0., 1.]])
tensor([[0.7129, 0.4326]], grad_fn=<AddmmBackward>)
tensor(0.4150, grad_fn=<MseLossBackward>)
Training 29/1412 done


tensor([[1., 0.]])
tensor([[0.5464, 0.4



tensor([[0., 1.]])
tensor([[-0.2241,  1.3070]], grad_fn=<AddmmBackward>)
tensor(0.0722, grad_fn=<MseLossBackward>)
Training 83/1412 done


KeyboardInterrupt: 

In [14]:
PATH = "/home/krys/POLIBUDA/nn_VoiceRecognition_Models/modelV5.pt"
torch.save(net.state_dict(), PATH)